# AutoML: Train "the best" Time-Series Forecasting model for the Energy Demand Dataset.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML time-series forecasting Job` with the 'forecasting()' factory-fuction.
- Train the model using AmlCompute by submitting/running the AutoML forecasting training job
- Obtaing the model and score predictions with it

**Motivations** - This notebook explains how to setup and run an AutoML forecasting job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'regression', 'classification', 'image classification', 'image object detection', 'nlp text classification', etc.

In this example we use the associated New York City energy demand dataset to showcase how you can use AutoML for a simple forecasting problem and explore the results. The goal is predict the energy demand for the next 48 hours based on historic time-series data.


# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient

from azure.ml._constants import AssetTypes
from azure.ml.automl import forecasting
from azure.ml.entities import JobInput

from pprint import pprint

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
#Enter details of your AML workspace

# SAGAR
subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91" #'<SUBSCRIPTION_ID>'
resource_group = "sasum_centraluseuap_rg" # '<RESOURCE_GROUP>'
workspace = "sasum-centraluseuap-ws" # '<AML_WORKSPACE_NAME>'

# CDLTLL
# subscription_id = '102a16c3-37d3-48a8-9237-4c9b1e8e80e0' #'<SUBSCRIPTION_ID>'
# resource_group = 'automlpmdemo' # '<RESOURCE_GROUP>'
# workspace = 'cesardl-automl-centraluseuap-ws' # '<AML_WORKSPACE_NAME>'

In [3]:
#get a handle to the workspace
credential = InteractiveBrowserCredential() # DefaultAzureCredential()
#credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace)

# 2. Data

We will use energy consumption [data from New York City](http://mis.nyiso.com/public/P-58Blist.htm) for model training. 
The data is stored in a tabular format and includes energy demand and basic weather data at an hourly frequency. 

With Azure Machine Learning MLTables you can keep a single copy of data in your storage, easily access data during model training, share data and collaborate with other users. 
Below, we will upload the data by creating an MLTable to be used for training.

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [13]:
# MLTable defined locally, with local data to be uploaded
my_training_data_input = JobInput(type=AssetTypes.MLTABLE, path="./mltable-folder")

# WITH REMOTE PATH
# my_training_data_input  = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my-forecasting-mltable")

# 3. Configure and run the AutoML Forecasting training job
In this section we will configure and run the AutoML job, for training the model.

## 3.1 Configure the job through the forecasting() factory function

### forecasting() function parameters:

The `forecasting()` factory function allows user to configure AutoML for the forecasting task for the most common scenarios with the following properties.

- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data', 'validation_data' and 'test_data'.
- `primary_metric` - The metric that AutoML will optimize for model selection.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g JobInput(mltable='my_mltable:1') OR JobInput(mltable=MLTable(local_path="./data"))
The parameter 'training_data' must always be provided.
- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it any other compute in the workspace. 
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `experiment_name` - The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.

### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.     
    
- timeout_minutes - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.

- trial_timeout_minutes - Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.
    
- max_trials - The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.
    
- max_concurrent_trials - Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.
    
- enable_early_termination - Whether to enable early termination if the score is not improving in the short term. 
    

## Specialized Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the .set_forecast_settings() method. 
The table below details the forecasting parameters we will be passing into our experiment.

|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**frequency**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.

# Advanced Forecasting Training <a id="advanced_training"></a>
### Using lags and rolling window features
This training is also using the **target lags**, that is the previous values of the target variables, meaning the prediction uses a horizon. We therefore must still specify the `forecast_horizon` that the model will learn to forecast. The `target_lags` keyword specifies how far back we will construct the lags of the target variable, and the `target_rolling_window_size` specifies the size of the rolling window over which we will generate the `max`, `min` and `sum` features.

This notebook uses the .set_training(blocked_models=) parameter to exclude some models that take a longer time to train on this dataset.  You can choose to remove models from the blocked_models list but you may need to increase the iteration_timeout_minutes parameter value to get results.

In [14]:
# Create the AutoML forecasting job with the related factory-function.

forecasting_job = forecasting(
                        compute = "cpu-cluster",
                        # name="dpv2-forecasting_job_01",
                        experiment_name = "dpv2-forecasting_experiment",
                        training_data = my_training_data_input,
                        target_column_name = "demand",
                        primary_metric = "NormalizedRootMeanSquaredError",
                        # (BUG 1711192) n_cross_validations=3,
                        enable_model_explainability=True,
                        tags={"owner": "cesardl"},
                        
                        # These are temporal properties needed in Private Preview
                        properties={
                            "_automl_internal_enable_mltable_quick_profile": True,
                            "_automl_internal_scenario": "non-prod",
                        }
)

# Limits are all optional
forecasting_job.set_limits(timeout = 600, #timeout_minutes
                           trial_timeout = 20, #trial_timeout_minutes
                           max_trials = 10, 
                           max_concurrent_trials = 4,
                           # max_cores_per_trial: -1,
                           enable_early_termination=True)

# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
            time_column_name="timeStamp",
            forecast_horizon=48,
            frequency="H",
            target_lags=[12],
            target_rolling_window_size=4,
            # ADDITIONAL FORECASTING TRAINING PARAMS ---
            # time_series_id_column_names=["tid1", "tid2", "tid2"],
            # short_series_handling_config=ShortSeriesHandlingConfiguration.DROP,
            # use_stl="season",
            # seasonality=3,
        )

# Training properties are optional
forecasting_job.set_training(blocked_models=["ExtremeRandomTrees"])

# Featurization properties are optional
# forecasting_job.set_featurization(# drop_columns=["not_needed_column"], # Optional
#                                   # enable_dnn_featurization=True         # Enable if there are text columns
#                                     )

# Visualization check
# Do not run this line if submitting the job, or it will fail...
# pprint(forecasting_job._to_rest_object().serialize())


## 2.2 Run the CommandJob
Using the `MLClient` created earlier, we will now run this CommandJob in the workspace.

In [15]:
# Submit the AutoML job (CDLTLL: Is it ml_client.create_or_update(forecasting_job))
returned_job = ml_client.jobs.create_or_update(forecasting_job)  # submit the job to the backend

print(f"Created job: {returned_job}")

Uploading mltable-folder (1.77 MBs): 100%|##########| 1771720/1771720 [00:02<00:00, 612514.99it/s]




Created job: ForecastingJob({'task_type': <TaskType.FORECASTING: 'Forecasting'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'display_name': None, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': '4f5af5c0-d5eb-4ca4-a39e-10502a861938', 'description': None, 'tags': {'owner': 'cesardl'}, 'properties': {'_automl_internal_enable_mltable_quick_profile': 'True', '_automl_internal_scenario': 'non-prod', 'mlflow.source.git.repoURL': 'git@github.com:Azure/azureml-examples.git', 'mlflow.source.git.branch': 'automl-preview', 'mlflow.source.git.commit': 'd09a72b7ce9781aa323b70f503aa094f1b1d5b31', 'azureml.git.dirty': 'True'}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sasum_centraluseuap_rg/providers/Microsoft.MachineLearningServices/workspaces/sasum-centraluseuap-ws/jobs/4f5af5c0-d5eb-4ca4-a39e-10502a861938', 'base_path': './', 'creation_context': <azure.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData ob

In [16]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/4f5af5c0-d5eb-4ca4-a39e-10502a861938?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sasum_centraluseuap_rg/workspaces/sasum-centraluseuap-ws&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'

## TO DO: Forecasting predictions and Test Metrics
This notebook needs to add the code for using the trained model by forecasting with a .py Script that can be run under the same Environment with a CommandJob to be defined.
After that inference logic, an evaluation of the test data inference showing metrics and a plotting chart should be added.

# Forecasting predictions<a id="forecast"></a>

Now we will retrieve the "best model" created by AutoML, so it can be used to make forecasting predictions on test data. We will score on the test dataset which should have the same schema as training dataset.

The inference will run on a remote compute. In this example, it will re-use the same training compute.

# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.